In [1]:
import pandas as pd
import numpy as np
import math
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output,State
from dash_table.Format import Format
import dash_table
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
from dash import Dash, html, dash_table, dcc, callback, Output, Input,callback_context
import plotly.express as px
import seaborn as sns
from sklearn.impute import SimpleImputer
import dash
from flask import Flask
import dash_bootstrap_components as dbc
from mlxtend.feature_selection import ExhaustiveFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from xgboost import XGBRegressor
import base64
import io
from category_encoders import *
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import fetch_openml
import statsmodels.api as sm
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from dash.exceptions import PreventUpdate
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
import re
import joblib
import optuna
import sys

sys.modules['sklearn.externals.joblib'] = joblib

C:\Users\leo\AppData\Local\Temp/ipykernel_30004/3761173232.py:7: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\leo\AppData\Local\Temp/ipykernel_30004/3761173232.py:8: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\leo\AppData\Local\Temp/ipykernel_30004/3761173232.py:10: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  from dash_table.Format import Format


In [2]:
class MyDash2:
    def __init__(self):
        #資料
        self.data_frame = None
        self.data_frame2 = None
        self.target_column=None
        
        #訓練和預測
        self.train_outcome=None
        self.result=None
        
        #防呆
        self.beforeclicks=None #之前點擊的次數
        self.page=1 #點的那一頁(預設第一頁)
        
        
    def placeholder_figure(self):
        placeholder_figure = go.Figure(data=[],
                                       layout=go.Layout(xaxis=dict(title='X Axis Title'),
                                                        yaxis=dict(title='Y Axis Title'),
                                                        margin=dict(l=40, r=40, t=10, b=20)))
        return placeholder_figure
    
    def run_dash(self):
        
        #sidebar的樣式
        SIDEBAR_STYLE = {
            "position": "fixed",
            "top": 0,
            "left": 0,
            "bottom": 0,
            "width": "16rem",
            "padding": "2rem 1rem",
            "background-color": "#f8f9fa",
            'font-size': '18px'
        }
        
        #內容的樣式
        CONTENT_STYLE = {
            "margin-left": "18rem", 
            "margin-right": "2rem",
            "padding": "2rem 1rem",
        }
        
        #sidebar
        sidebar = html.Div([
            dbc.Nav([
                dbc.NavLink('Current Data',href='/page0',id='link_0'),
                html.Hr(className='my-2', style={'border-top': '1.5px solid rgb(125, 125, 125)'}),  
                dbc.Accordion(
                    [
                        dbc.AccordionItem(
                            [dbc.Nav([
                                dbc.NavLink('Data Uploading',href='/page1',id='link_1'),
                                dbc.NavLink('Data EDA', href='/page2',id='link_2'),
                                dbc.NavLink('Missing Value Imputing',href='/page3',id='link_3'),
                                ], vertical=True, pills=True, navbar=True,style={'marginLeft':'20px'}),],title='Data Preparation'),
                        dbc.AccordionItem(
                            [dbc.Nav([
                                dbc.NavLink('Encoding',href='/page4',id='link_4'),
                                dbc.NavLink('Standardization', href='/page5',id='link_5'),
                                ], vertical=True, pills=True, navbar=True,style={'marginLeft':'20px'}),],title='Feature Engineering'),
                        dbc.AccordionItem(
                            [dbc.Nav([
                                dbc.NavLink('Introduction',href='/page6',id='link_6'),
                                dbc.NavLink('Filter Method', href='/page7',id='link_7'),
                                dbc.NavLink('Wrapper Method', href='/page8',id='link_8'),
                                dbc.NavLink('Embedded Method', href='/page9',id='link_9'),
                                ], vertical=True, pills=True, navbar=True,style={'marginLeft':'20px'}),],title="Feature Selection"),
                        dbc.AccordionItem(
                            [dbc.Nav([
                                dbc.NavLink('Prediction', href='/page10',id='link_10'),
                                ], vertical=True, pills=True, navbar=True,style={'marginLeft':'20px'}),],title="Prediction")
                    ],flush=True,start_collapsed=True),
                html.Hr(style={'border-top': '1.5px solid rgb(125, 125, 125)'}), 
            ],
            vertical=True,
            pills=True,
            ),
        ],
        style=SIDEBAR_STYLE)

    
        modal = html.Div([
            dbc.Modal([
                dbc.ModalHeader(
                    dbc.ModalTitle("Reminding"), close_button=False),
                    dbc.ModalBody(
                        "This part you have done already "
                        "Try clicking the below close button."
                    ),
                    dbc.ModalFooter(dbc.Button("Close", id="close-dismiss")),],
                    id="modal-dismiss",
                    keyboard=False,
                    backdrop="static")])
        
        
        
        content = html.Div(id="page-content", children=[], style=CONTENT_STYLE)
        
        
        app = Dash(__name__,
                external_stylesheets=[dbc.themes.MINTY],
                suppress_callback_exceptions=True
        )
        
        app.layout = html.Div([
            modal,
            dcc.Location(id='url', refresh=False), #獲取當前的URL路徑，以根據URL路徑顯示相應的內容。 
            sidebar,
            content
        ])
        
        LINK_0_STYLE = {"color": "rgb(178, 136, 102)"} 
        DEFAULT_STYLE = {"color": "rgb(147, 202, 173)"} 
        ACTIVE_STYLE = {"color": "rgb(238, 145, 125)"} 

        @app.callback(
            [Output(f"link_{i}", "style") for i in range(11)],
            [Input(f"link_{i}", "n_clicks") for i in range(11)],
            State("url", "pathname")
        )
        def highlight_links(*args):
            
            n_clicks = args[:-1]
            pathname = args[-1]
            styles = [LINK_0_STYLE] + [DEFAULT_STYLE] * 10
            for i, _ in enumerate(n_clicks[1:], 1):
                active = re.search(r"page(\d+)", pathname)  
                if active and int(active.group(1)) == i:
                    styles[i] = ACTIVE_STYLE
                elif n_clicks[i] : 
                    styles[i] = {"color": "rgb(100, 100, 100)"}
            return styles
        
        @app.callback(
            Output("modal-dismiss", "is_open"),
            [Input(f"link_{i}", "n_clicks") for i in range(11)],
            Input("close-dismiss", "n_clicks"),
            State("modal-dismiss", "is_open"),
        )
        def poke(*args):
            n_clicks = args[:-2]
            n_close=args[-2]
            is_open=args[-1]
            n_clicks=list(n_clicks)
            for i in range(len(n_clicks)):
                if n_clicks[i] is None:
                    n_clicks[i]=0
            n_clicks=tuple(n_clicks)
            
            if self.beforeclicks is not None:
                for i in range(len(self.beforeclicks)):
                    if n_clicks[i]!=self.beforeclicks[i]:
                        self.page=i
            self.beforeclicks=n_clicks
            
            if self.page ==0: 
                return is_open
            elif self.page ==2: 
                return is_open
            elif self.page ==6: 
                return is_open
            elif self.page ==10: 
                return is_open
            
            if n_clicks[self.page] is not None and n_clicks[self.page]>1:
                return not is_open
            elif n_clicks[self.page] or n_close:
                return is_open
            else:
                return is_open

        
# =======================================================================================================================

        #第零頁 檔案現況
        page0_layout = html.Div(
            children=[
                html.Br(),
                html.H1("Current Data",id='current'),
                html.Br(),
                #train data
                html.H3('Train Data'),
                html.Hr(),
                html.Br(),
                html.Div(id='train-data-upload-result0',style={'text-align': 'center'}),
                html.Br(),
                #test data
                html.H3('Test Data'),
                html.Hr(),
                html.Br(),
                html.Div(id='test-data-upload-result0',style={'text-align': 'center'}),
            ],
            style={'text-align': 'center'}
        )
        
        @app.callback(
        Output('train-data-upload-result0', 'children'),
        Input('current', 'children'),
        )
        
        def store_data_train(current):
            if self.data_frame is not None :
                return  dash_table.DataTable(
                        data=self.data_frame.to_dict('records'),
                        columns=[{'name': col, 'id': col} for col in self.data_frame.columns],
                        page_size=10,style_table={'overflowX': 'auto'})
            else:
                return html.H5("You haven't uploaded train data yet.",style={'color': "red"})


        @app.callback(
        Output('test-data-upload-result0', 'children'),
        Input('current', 'children'),
        )
        
        def store_data_test(current):
            if self.data_frame2 is not None :
                return  dash_table.DataTable(
                        data=self.data_frame2.to_dict('records'),
                        columns=[{'name': col, 'id': col} for col in self.data_frame2.columns],
                        page_size=10,style_table={'overflowX': 'auto'})
            else:
                return html.H5("You haven't uploaded test data yet.",style={'color': "red"})


# =======================================================================================================================

        #第一頁 讀檔案
        page1_layout = html.Div(
            children=[
                html.Br(),
                html.H1("Data preparation"),
                html.Br(),
                html.H3("Upload your data"),
                html.Br(),
                #train data
                dcc.Upload(
                    id='upload-data-train',
                    children=html.Div([
                        'Train: Drag and Drop or ',
                        html.A('Select Files')
                    ]),
                    style={
                        'width': '100%',
                        'height': '60px',
                        'lineHeight': '60px',
                        'borderWidth': '1px',
                        'borderStyle': 'dashed',
                        'borderRadius': '5px',
                        'textAlign': 'center',
                        'margin': '10px'
                    },
                    multiple=True
                ),
                
                html.H5(id='train-data-upload-result',style={'text-align': 'center'}),
                html.Br(),
               
                #test data
                dcc.Upload(
                    id='upload-data-test',
                    children=html.Div([
                        'Test: Drag and Drop or ',
                        html.A('Select Files')
                    ]),
                    style={
                        'width': '100%',
                        'height': '60px',
                        'lineHeight': '60px',
                        'borderWidth': '1px',
                        'borderStyle': 'dashed',
                        'borderRadius': '5px',
                        'textAlign': 'center',
                        'margin': '10px'
                    },
                    multiple=True),
                html.H5(id='test-data-upload-result',style={'text-align': 'center'}),
                html.Br(),
                
                #target chosen
                html.H3('Choose a variable to be a target',style={'text-align': 'center'}),
                dcc.Dropdown(id="column_target", 
                                multi=False,
                                style={'text-align': 'center'}),
                html.Br(),
                html.Div([html.Button('confirm', id='button0',n_clicks=0,style={'text-align': 'center','background-color': 'rgb(190, 223, 210)'})],
                          style={'textAlign': 'center', 'position': 'relative'}),
                html.Br(),
                html.H3(id='text',style={'text-align': 'center'}),
                html.Br(),
            ],
            style={'text-align': 'center'}
        )
        
        @app.callback(
        Output('train-data-upload-result', 'children'),
        Input('upload-data-train', 'contents'),
        )
        
        def store_data_train(train_contents):
            if train_contents is not None:
                train_content_type, train_content_string = train_contents[0].split(',')
                decoded = base64.b64decode(train_content_string)
                self.data_frame= pd.read_csv(io.StringIO(decoded.decode('utf-8')))
                self.option_train=self.data_frame.columns
                return  html.H5('Train data has been upload.',style={'text-align': 'center','color':'green'})
            elif self.data_frame is not None:
                return  html.H5('Train data exist! you can renew your data',style={'text-align': 'center','color':'green'})
            else:
                return "Please Upload Train Data"


        @app.callback(
        Output('test-data-upload-result', 'children'),
        Input('upload-data-test', 'contents'),
        )
        
        def store_data_test(test_contents):
            if test_contents is not None:
                test_content_type, test_content_string = test_contents[0].split(',')
                decoded = base64.b64decode(test_content_string)
                self.data_frame2 = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
                self.option_test=self.data_frame2.columns
                return  html.H5('Test data has been upload.',style={'text-align': 'center','color':'green'})
            elif self.data_frame2 is not None:
                return  html.H5('Test data exist! you can renew your data',style={'text-align': 'center','color':'green'})
            else:
                return "Please Upload Test Data" 
        @app.callback(
            Output("column_target", "options"),
            Input('train-data-upload-result', "children")
        )
        def get_options(useless0):
            output = {}
            if self.data_frame is not None:
                dff = self.data_frame
                output = dff.columns
            return  output
           
        @app.callback(
            Output("text", "children"),
            Input("column_target", "value"),
            Input(component_id='button0', component_property='n_clicks') 
        )  
        def show(value,n_clicks):
            if n_clicks:
                self.target_column=value
                return html.H5(f'the target you select is {self.target_column}')
            else:
                if self.target_column is not None:
                    return html.H5(f'the target you select is {self.target_column}')
                else:
                    return html.H5(f"you haven't choose a target",style={'color': "red"})
                

        
# =======================================================================================================================

        #page3 EDA
        page2_layout=([
            html.Br(),
            html.H1("Data EDA",style={'text-align': 'center'}),
            html.Br(),
            dcc.RadioItems(id="EDA-radioitems",
                           options=["Train", "Test"], value="Train",
                           labelStyle={'display': 'inline-block', 'margin-right': '10px'}),
            html.Div([
                html.Div(children=[
                        html.H3("distribution visualization",style={'text-align': 'center'}),
                        html.Br(),
                        html.H5("choose a column",style={'text-align': 'center'}),
                        dcc.Dropdown(id="slt_feat1", 
                                    multi=False,
                                    style={'text-align': 'center'}),
                        html.Br(),
                        html.Br(),
                        dcc.Graph(id="Single_feat"),
                        html.Div(id="nullcounts")], 
                        style={'width':'50%', 'padding':'1%'}),
                html.Div(children=[
                        html.H3("comparation of two variable",style={'text-align': 'center'}),
                        html.Br(),
                        html.H5("choose your first variable",style={'text-align': 'center'}),
                        dcc.Dropdown(id="slt_feat2", 
                                    multi=False,
                                    style={'text-align': 'center'}),

                        html.H5("choose your second variable",style={'text-align': 'center'}),
                        dcc.Dropdown(id="slt_feat3", 
                                    multi=False, 
                                    style={'text-align': 'center'}),
                        dcc.Graph(id="Double_feat")], 
                        style={'width':'50%', 'padding':'1%'})
            ], style={
                    'display':'flex',
                    'align-item':'center',
                    'justify-content':'space-evenly',
                    'flex-direction':'row' }),
            
            # Train test comparison
            html.Div(children=[
                html.Br(),
                html.H3(id='useless',children='Train Test Comparison',style={'text-align': 'center'}),
                html.Br(),
                dcc.Dropdown(id="train_test_dropdown",
                             multi=False,
                             style={'text-align': 'center'})]),
            html.Br(),
            html.Div(children=[
                html.Div([
                    dcc.Graph(id="train_test_comparison")],
                    style={'width':'50%', 'padding':'1%'}),
                html.Br(),
                html.Div([
                    dcc.Graph(id="train_test_comparison2")],
                    style={'width':'50%', 'padding':'1%'})
            ], style={
                    'display':'flex',
                    'align-item':'center',
                    'justify-content':'space-evenly',
                    'flex-direction':'row' })
         ])
        

        @app.callback(
            Output("slt_feat1", "options"),
            Output("slt_feat2", "options"),
            Output('slt_feat3','options'),
            Input("EDA-radioitems", "value")
        )
        
        def update_EDA_options(option):
            output = [{"label": "Ex1", "value": 'None'}]
            if option == "Train":
                if self.data_frame is not None:
                    output = self.data_frame.columns
            else:
                if self.data_frame2 is not None:
                    output = self.data_frame2.columns
            return output,output,output
            
        @app.callback(
            Output(component_id='Single_feat', component_property='figure'),
            Input(component_id='slt_feat1', component_property='value'),
            Input(component_id="EDA-radioitems", component_property="value")
        )    
        #若有任何一個引數為None則回傳True
        def update(feat1, tset):
            def anyNone(*args):
                for i in args:
                    if i is None:
                        return True 
                return False
            fig1 = self.placeholder_figure()
            target = self.target_column
            targetdf = None if (tset is None) else (self.data_frame if tset == 'Train' else self.data_frame2)
            if not anyNone(targetdf, feat1) and feat1 in targetdf.columns: 
                # 類別型資料做 barplot
                if (len(set(targetdf.loc[:,feat1]))<=4 or targetdf.dtypes[feat1] in ["object"]):
                    counts = targetdf.loc[:,feat1].value_counts()
                    fig1 = px.bar(counts,x=counts.index,y=counts.values,title=f'barplot',color_discrete_sequence=['rgb(160, 198, 213)'])
                # 數值型資料做 histogram
                else:
                    fig1 = px.histogram(targetdf, x=feat1, title=f'Histogram',color_discrete_sequence=['rgb(160, 198, 213)'])
                    fig1.update_traces(marker=dict(line=dict(color='rgb(100,100,100)', width=0.8)))
                    
            return fig1
                    
        @app.callback(
            Output(component_id='Double_feat', component_property='figure'),
            Input(component_id='slt_feat2', component_property='value'),
            Input(component_id='slt_feat3', component_property='value'),
            Input(component_id="EDA-radioitems", component_property="value")
        )
        
        def update(feat2, feat3, tset):
            def anyNone(*args):
                for i in args:
                    if i is None:
                        return True
                return False
            fig2 = self.placeholder_figure()
            target = self.target_column
            targetdf = None if anyNone(self.data_frame,self.data_frame2) else (self.data_frame if tset=='Train' else self.data_frame2)
            if not anyNone(targetdf, feat2,feat3) and feat2 in targetdf.columns and feat3 in targetdf.columns: 
                # 數值對類別做 boxplot，數值對數值做 scatter 類別對類別做barplot
                if ((len(set(targetdf.loc[:,feat2]))<=4 or targetdf.dtypes[feat2] in ["object"]) and len(set(targetdf.loc[:,feat3]))>4):
                    fig2 = px.box(targetdf,x=feat2, y=feat3,title='box plot',
                                  color_discrete_sequence=['rgb(160, 198, 213)'])
                elif (len(set(targetdf.loc[:,feat2]))>4 and (len(set(targetdf.loc[:,feat3]))<=4 or targetdf.dtypes[feat2] in ["object"])):
                    fig2 = px.box(targetdf,x=feat2, y=feat3,title='box plot',
                                  color_discrete_sequence=['rgb(160, 198, 213)'])
                elif (len(set(targetdf.loc[:,feat2]))<=4 and (len(set(targetdf.loc[:,feat3]))<=4)) or (targetdf.dtypes[feat2] in ["object"] and targetdf.dtypes[feat3] in ["object"]):
                    fig2 = px.histogram(targetdf, x=feat2,color=feat3,barmode='group',color_discrete_sequence=['rgb(160, 198, 213)','rgb(160, 198, 184)','rgb(255,222,173)','rgb(188,143,143)'])
                else:
                    fig2 = px.scatter(targetdf, x=feat2, y=feat3, title="Scatter Plot",
                                      color_discrete_sequence=['rgb(160, 198, 213)'],
                                      trendline='ols',
                                      trendline_color_override='orange')
            return fig2


        @app.callback(
        Output("train_test_dropdown",'options'),
        Input('useless','children'))
        
        def get_options(useless):
            output = [{"label": "Ex1", "value": 'None'}]
            if self.data_frame2 is not None:
                output = self.data_frame2.columns
            return output
        
        @app.callback(
            Output('train_test_comparison','figure'),
            Output('train_test_comparison2','figure'),
            Input("train_test_dropdown",'value')
        )
        def train_test_comparison(train_test_dropdown):
            def anyNone(*args):
                for i in args:
                    if i is None:
                        return True
                return False
            fig10, fig11 = self.placeholder_figure(), self.placeholder_figure()
            df_1 = self.data_frame
            df_2 = self.data_frame2
            if not anyNone(df_1,df_2,train_test_dropdown) and train_test_dropdown in df_2.columns: 
                x1 = df_1.loc[:,train_test_dropdown]
                x2 = df_2.loc[:,train_test_dropdown]
                df1 = pd.DataFrame({"index": x1.index, f"{train_test_dropdown}": x1, "Type": "train"})
                df2 = pd.DataFrame({"index": x2.index+x1.shape[0], f"{train_test_dropdown}": x2, "Type": "test"})
                df = pd.concat([df1, df2])
                if len(set(df_1.loc[:,train_test_dropdown]))>4 and len(set(df_2.loc[:,train_test_dropdown]))>4:
                    fig10 = px.scatter(df, x='index',y=f'{train_test_dropdown}',color='Type',color_discrete_sequence=['rgb(160, 198, 213)','rgb(160, 198, 184)'],title=f'trend of {train_test_dropdown} for train and test set')
                    fig11=px.box(df,x='Type',y=f'{train_test_dropdown}',color_discrete_sequence=['rgb(160, 198, 213)','rgb(160, 198, 184)'],title=f'{train_test_dropdown} in train and test')
                else :
                    train_counts = x1.value_counts()
                    test_counts = x2.value_counts()
                    combined_counts = pd.concat([train_counts, test_counts], axis=1)
                    combined_counts.columns = ['Train', 'Test']
                    fig10 = px.bar(combined_counts, x=combined_counts.index, y=['Train', 'Test'], barmode='group',
                                   color_discrete_sequence=['rgb(160, 198, 213)','rgb(160, 198, 184)'],title=f'count in train and test')
                    fig10.update_layout(yaxis=dict(title='Counts'))
            
            return fig10,fig11
           
# =======================================================================================================================

        page3_layout = html.Div(
            children=[
                html.Br(),
                html.H1("Missing Value Imputing",style={'text-align': 'center'}),
                
                #畫所有資料的缺失量(histogram)
                
                #train data
                html.Br(),
                html.Br(),
                html.H3("Histogram of missing values in train data",style={'text-align': 'center'}),
                dcc.Graph(id="hist_missing",style={'display': 'block'}),
                html.Br(),
                html.Div(id="nullpercent",
                         style={'font-size':'18px','display': 'block','text-align': 'center'}),
                html.Br(),
                html.Br(),
                
                #test data
                html.Br(),
                html.H3("Histogram of missing values in test data",style={'text-align': 'center'}),
                dcc.Graph(id="hist_missing2",style={'display': 'block'}),
                html.Br(),
                html.Div(id="nullpercent2",
                         style={'font-size':'18px','display': 'block','text-align': 'center'}),
                html.Br(),
                html.Br(),

                html.H4("How to impute categorical variables (required)",style={'text-align': 'left'}), 
                html.Br(),
                dcc.Dropdown(id="dealwith_categ", options=[{'label':'Drop NaN','value':'Drop NaN'},{'label':'Mode','value':'Mode'}], 
                             multi=False,
                             value=None,
                             style={'width': "90%"}),
                
                
                html.Br(),
                
                #option
                html.H4("How to impute numeric variables (required)",style={'text-align': 'left'}), 
                html.Br(),
                dcc.Dropdown(id="imputation", options=[
                                                       {'label':'Drop NaN','value':'Drop NaN'},
                                                       {'label':'Mode','value':'Mode'},
                                                       {'label':'Median','value':'Median'},
                                                       {'label':'Mean','value':'Mean'},
                                                       {'label':'KNN-3','value':'KNN-3'},
                                                       {'label':'KNN-5','value':'KNN-5'}], 
                             multi=False,
                             value=None, #預設值
                             style={'width': "70%"}),
                
                html.Br(),
                html.Div([html.Button('confirm', id='button1',n_clicks=0,style={'text-align': 'center','background-color': 'rgb(190, 223, 210)'})],
                      style={'textAlign': 'center', 'position': 'relative'}),
                
                #更新補值後資料
                html.Br(),
                html.H3("Now your data:",style={'text-align': 'center'}), 
                html.Br(),
                html.H5("Train data:",style={'text-align': 'center'}), 
                html.Div(id="data_page"),
                html.Br(),
                html.Br(),
                html.H5("Test data:",style={'text-align': 'center'}), 
                html.Div(id="data_page_test")
            ]
        )
                
        @app.callback(
            Output(component_id='hist_missing', component_property='figure'),
            Output(component_id='nullpercent', component_property='children'), 
            Output(component_id='hist_missing2', component_property='figure'), 
            Output(component_id='nullpercent2', component_property='children'),  
            Output(component_id='data_page', component_property='children'),
            Output(component_id='data_page_test', component_property='children'),
            Input(component_id='imputation', component_property='value'),
            Input(component_id='dealwith_categ', component_property='value'),
            Input(component_id='button1',component_property='n_clicks')
        )

        def update_plotp3(imputation,dealwith_categ,n_clicks):
            fig5 = self.placeholder_figure()
            nullper_text = ''
            fig9 = self.placeholder_figure()
            nullper_text2 = ''
            table_train = html.H5(f"you haven't upload train data",style={'text-align': 'center','color': "red"})
            table_test = html.H5(f"you haven't upload test data",style={'text-align': 'center','color': "red"})
            if self.data_frame is not None and self.data_frame2 is not None:
                missing_values = self.data_frame.isna().sum().sort_values(ascending=False)
                missing_values2 = self.data_frame2.isna().sum().sort_values(ascending=False)
                if missing_values.sum()==0 and missing_values2.sum()==0:
                    table_train=dash_table.DataTable(data=self.data_frame.to_dict('records'),
                                         page_size=10,style_table={'overflowX': 'auto'})
                    table_test=dash_table.DataTable(data=self.data_frame2.to_dict('records'),
                                         page_size=10,style_table={'overflowX': 'auto'})
                else:
                    fig5 = px.histogram(x=missing_values.index, y=missing_values.values,color_discrete_sequence=['rgb(160, 198, 213)'])
                    fig5.update_traces(marker=dict(line=dict(color='black', width=0.5))) #加框線

                    #回傳缺失值比例
                    missing_per = 100*round(self.data_frame.isnull().sum().sum()/(self.data_frame.shape[0]*self.data_frame.shape[1]),3)
                    nullper_text = f"Percentage of missing values in train set: {missing_per} %"

                    
                    fig9 = px.histogram(x=missing_values2.index, y=missing_values2.values,color_discrete_sequence=['rgb(160, 198, 184)'])
                    fig9.update_traces(marker=dict(line=dict(color='black', width=0.5))) #加框線

                    #回傳缺失值比例
                    missing_per2 = 100*round(self.data_frame2.isnull().sum().sum()/(self.data_frame2.shape[0]*self.data_frame2.shape[1]),3)
                    nullper_text2 = f"Percentage of missing values in test set: {missing_per2} %"

                    if n_clicks:
                        #處理類別型資料的方式
                        if dealwith_categ=='Mode':
                            categ_list=[i for i in self.data_frame.columns if self.data_frame.dtypes[i] in ['object'] or len(set(self.data_frame.loc[:,i]))<=4]
                            for feat in categ_list:
                                arg = np.array(self.data_frame[feat])
                                arg = arg.reshape([-1, 1])
                                imr = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
                                imr = imr.fit(arg)
                                imputed_data = imr.transform(arg)
                                self.data_frame[feat] = imputed_data
                            categ_list2=[i for i in self.data_frame2.columns if self.data_frame2.dtypes[i] in ['object'] or len(set(self.data_frame2.loc[:,i]))<=4]
                            for feat in categ_list2:
                                arg = np.array(self.data_frame2[feat])
                                arg = arg.reshape([-1, 1])
                                imr = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
                                imr = imr.fit(arg)
                                imputed_data = imr.transform(arg)
                                self.data_frame2[feat] = imputed_data
                        elif dealwith_categ=='Drop NaN':
                            categ_list=[i for i in self.data_frame.columns if self.data_frame.dtypes[i] in ['object'] or len(set(self.data_frame.loc[:,i]))<=4]
                            self.data_frame=self.data_frame.dropna(subset=categ_list).reset_index(drop=True)
                            self.data_frame2=self.data_frame2.dropna(subset=categ_list).reset_index(drop=True)

                        if imputation=='Drop NaN':
                            self.data_frame=self.data_frame.dropna().reset_index(drop=True)
                            self.data_frame2=self.data_frame2.dropna().reset_index(drop=True)
                            nullper_text = f"Percentage of missing values in train set: 0 %"
                            nullper_text2 = f"Percentage of missing values in test set: 0 %"
                            missing_values = self.data_frame.isna().sum()
                            fig5 = px.histogram(x=missing_values.index, y=missing_values.values,color_discrete_sequence=['rgb(160, 198, 213)'])
                            missing_values2 = self.data_frame2.isna().sum()
                            fig9 = px.histogram(x=missing_values2.index, y=missing_values2.values,color_discrete_sequence=['rgb(160, 198, 184)'])

                        elif imputation == 'Mean':
                            has_missing_values = self.data_frame.isna().any(axis=0)  #每個欄位有沒有缺失值(總欄位數個True或False)
                            cols_with_missing_values = self.data_frame.columns[has_missing_values]  #取出哪些欄位有缺失值
                            for feat in cols_with_missing_values:
                                arg = np.array(self.data_frame[feat])  
                                arg = arg.reshape([-1, 1])   

                                imr = SimpleImputer(missing_values=np.nan, strategy='mean')
                                imr = imr.fit(arg)
                                imputed_data = imr.transform(arg)
                                self.data_frame[feat] = imputed_data

                            has_missing_values2 = self.data_frame2.isna().any(axis=0)  
                            cols_with_missing_values2 = self.data_frame2.columns[has_missing_values2] 
                            for feat in cols_with_missing_values2:
                                arg = np.array(self.data_frame2[feat]) 
                                arg = arg.reshape([-1, 1])  

                                imr = SimpleImputer(missing_values=np.nan, strategy='mean')
                                imr = imr.fit(arg)
                                imputed_data = imr.transform(arg) 
                                self.data_frame2[feat] = imputed_data
                            nullper_text = f"Percentage of missing values in train set: 0 %"
                            nullper_text2 = f"Percentage of missing values in test set: 0 %" 
                            missing_values = self.data_frame.isna().sum()
                            fig5 = px.histogram(x=missing_values.index, y=missing_values.values,color_discrete_sequence=['rgb(160, 198, 213)'])
                            missing_values2 = self.data_frame2.isna().sum()
                            fig9 = px.histogram(x=missing_values2.index, y=missing_values2.values,color_discrete_sequence=['rgb(160, 198, 184)'])

                        elif imputation == 'Mode' : 
                            has_missing_values = self.data_frame.isna().any(axis=0)
                            cols_with_missing_values = self.data_frame.columns[has_missing_values]
                            for feat in cols_with_missing_values:
                                arg = np.array(self.data_frame[feat])
                                arg = arg.reshape([-1, 1])

                                imr = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
                                imr = imr.fit(arg)
                                imputed_data = imr.transform(arg)
                                self.data_frame[feat] = imputed_data

                            has_missing_values2 = self.data_frame2.isna().any(axis=0)  
                            cols_with_missing_values2 = self.data_frame2.columns[has_missing_values2] 
                            for feat in cols_with_missing_values2:
                                arg = np.array(self.data_frame2[feat]) 
                                arg = arg.reshape([-1, 1])  

                                imr = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
                                imr = imr.fit(arg)
                                imputed_data = imr.transform(arg) 
                                self.data_frame2[feat] = imputed_data
    
                            nullper_text = f"Percentage of missing values in train set: 0 %"
                            nullper_text2 = f"Percentage of missing values in test set: 0 %" 
                            missing_values = self.data_frame.isna().sum()
                            fig5 = px.histogram(x=missing_values.index, y=missing_values.values,color_discrete_sequence=['rgb(160, 198, 213)'])
                            missing_values2 = self.data_frame2.isna().sum()
                            fig9 = px.histogram(x=missing_values2.index, y=missing_values2.values,color_discrete_sequence=['rgb(160, 198, 184)'])


                        elif imputation == 'Median':
                            has_missing_values = self.data_frame.isna().any(axis=0)
                            cols_with_missing_values = self.data_frame.columns[has_missing_values]
                            for feat in cols_with_missing_values:
                                arg = np.array(self.data_frame[feat])
                                arg = arg.reshape([-1, 1])

                                imr = SimpleImputer(missing_values=np.nan, strategy='median')
                                imr = imr.fit(arg)
                                imputed_data = imr.transform(arg)
                                self.data_frame[feat] = imputed_data

                            has_missing_values2 = self.data_frame2.isna().any(axis=0)  
                            cols_with_missing_values2 = self.data_frame2.columns[has_missing_values2] 
                            for feat in cols_with_missing_values2:
                                arg = np.array(self.data_frame2[feat]) 
                                arg = arg.reshape([-1, 1])  

                                imr = SimpleImputer(missing_values=np.nan, strategy='median')
                                imr = imr.fit(arg)
                                imputed_data = imr.transform(arg) 
                                self.data_frame2[feat] = imputed_data
                            nullper_text = f"Percentage of missing values in train set: 0 %"
                            nullper_text2 = f"Percentage of missing values in test set: 0 %"      
                            missing_values = self.data_frame.isna().sum()
                            fig5 = px.histogram(x=missing_values.index, y=missing_values.values,color_discrete_sequence=['rgb(160, 198, 213)'])
                            missing_values2 = self.data_frame2.isna().sum()
                            fig9 = px.histogram(x=missing_values2.index, y=missing_values2.values,color_discrete_sequence=['rgb(160, 198, 184)'])

                        elif imputation == "KNN-3":
                            has_missing_values = self.data_frame.isna().any(axis=0)  #每個欄位有沒有缺失值(總欄位數個True或False)
                            cols_with_missing_values = self.data_frame.columns[has_missing_values]  #取出哪些欄位有缺失值
                            for feat in cols_with_missing_values:
                                arg = np.array(self.data_frame[feat])
                                arg = arg.reshape([-1, 1])  

                                imr = KNNImputer(n_neighbors=3)
                                imr = imr.fit(arg)
                                imputed_data = imr.transform(arg) 
                                self.data_frame[feat] = imputed_data.reshape(-1)

                            has_missing_values2 = self.data_frame2.isna().any(axis=0)  
                            cols_with_missing_values2 = self.data_frame2.columns[has_missing_values2] 
                            for feat in cols_with_missing_values2:
                                arg = np.array(self.data_frame2[feat]) 
                                arg = arg.reshape([-1, 1])  

                                imr = KNNImputer(n_neighbors=3)
                                imr = imr.fit(arg)
                                imputed_data = imr.transform(arg) 
                                self.data_frame2[feat] = imputed_data.reshape(-1)
                            
                            nullper_text = f"Percentage of missing values in train set: 0 %"
                            nullper_text2 = f"Percentage of missing values in test set: 0 %"    
                            missing_values = self.data_frame.isna().sum()
                            fig5 = px.histogram(x=missing_values.index, y=missing_values.values,color_discrete_sequence=['rgb(160, 198, 213)'])
                            missing_values2 = self.data_frame2.isna().sum()
                            fig9 = px.histogram(x=missing_values2.index, y=missing_values2.values,color_discrete_sequence=['rgb(160, 198, 184)'])

                        elif imputation == "KNN-5":
                            has_missing_values = self.data_frame.isna().any(axis=0)  #每個欄位有沒有缺失值(總欄位數個True或False)
                            cols_with_missing_values = self.data_frame.columns[has_missing_values]  #取出哪些欄位有缺失值
                            for feat in cols_with_missing_values:
                                arg = np.array(self.data_frame[feat])
                                arg = arg.reshape([-1, 1])  

                                imr = KNNImputer(n_neighbors=5)
                                imr = imr.fit(arg)
                                imputed_data = imr.transform(arg) 
                                self.data_frame[feat] = imputed_data.reshape(-1)

                            has_missing_values2 = self.data_frame2.isna().any(axis=0)  
                            cols_with_missing_values2 = self.data_frame2.columns[has_missing_values2] 
                            for feat in cols_with_missing_values2:
                                arg = np.array(self.data_frame2[feat]) 
                                arg = arg.reshape([-1, 1])  

                                imr = KNNImputer(n_neighbors=5)
                                imr = imr.fit(arg)
                                imputed_data = imr.transform(arg) 
                                self.data_frame2[feat] = imputed_data.reshape(-1)
                            
                            nullper_text = f"Percentage of missing values in train set: 0 %"
                            nullper_text2 = f"Percentage of missing values in test set: 0 %"
                            missing_values = self.data_frame.isna().sum()
                            fig5 = px.histogram(x=missing_values.index, y=missing_values.values,color_discrete_sequence=['rgb(160, 198, 213)'])
                            missing_values2 = self.data_frame2.isna().sum()
                            fig9 = px.histogram(x=missing_values2.index, y=missing_values2.values,color_discrete_sequence=['rgb(160, 198, 184)'])

                        table_train = dash_table.DataTable(data=self.data_frame.to_dict('records'),
                                         page_size=10,style_table={'overflowX': 'auto'})
                        table_test = dash_table.DataTable(data=self.data_frame2.to_dict('records'),
                                         page_size=10,style_table={'overflowX': 'auto'})
                    else:
                        table_train=dash_table.DataTable(data=self.data_frame.to_dict('records'),
                                         page_size=10,style_table={'overflowX': 'auto'})
                        table_test=dash_table.DataTable(data=self.data_frame2.to_dict('records'),
                                         page_size=10,style_table={'overflowX': 'auto'})
                        
            return fig5,nullper_text,fig9,nullper_text2,table_train,table_test
            

# =======================================================================================================================
        
        #encoding
        page4_layout=html.Div(
        children=[
            html.Br(),
            html.H1(children="Feature Engineering",style={'text-align': 'center'}),
            html.Hr(),
            html.Br(),
            html.H3(id='useless2',children="Encode Categorical Variables",style={'text-align': 'center'}),
            html.Hr(),
            html.Br(),
            html.Div(id='warn_encode',style={'text-align': 'center'}),
            html.Br(),
            dcc.Checklist(id="columns", 
                            inline=True,
                            labelStyle={"display": "inline-block", "margin-right": "12px"},
                            style={'text-align': 'center'}),
            html.Br(),
            html.H5('Encoding Options',style={'text-align': 'center'}),
            dcc.Dropdown(id="encode_options",
                            options=['one-hot encoding','target encoding','label encoding'],
                            multi=False,
                            value=[],
                            ),
            html.Br(),
            html.Br(),
            html.Div([html.Button('confirm', id='button',n_clicks=0,style={'text-align': 'center','background-color': 'rgb(190, 223, 210)'})],
                      style={'textAlign': 'center', 'position': 'relative'}),
            html.Div(id='false',style={'text-align': 'center'}),
            html.Br(),
            html.Div(id='success',style={'text-align': 'center'}),
        ])
        
        @app.callback(
            Output("warn_encode", "children"),
            Output("columns", "options"),
            Input("useless2", "children")
        )
        
        def get_options(useless2):
            output2 = {}
            warn=html.H5('You have not upload your data',style={'color': "red"})
            if self.data_frame is not None:
                dff = self.data_frame
                output2 = [i for i in dff.columns if dff.dtypes[i] in ['object']]
                if output2==[]:
                    warn=html.H5('No categorical variable need to encode !!',style={'color': "red"})
                    return warn,output2
                else:
                    warn=html.H5('These are catagorical variables in your data !!')
                    return warn,output2
            else:
                return warn,output2
        
        @app.callback(
        [Output(component_id='success',component_property='children'),            
        Output(component_id='false',component_property='children')], 
        [Input(component_id='columns',component_property='value'),
        Input(component_id='encode_options',component_property='value'),
        Input(component_id='button', component_property='n_clicks')]
        )
        
        def encode(column,encode_options,n_clicks):
            false = ''
            result={}
            if n_clicks:
                if self.data_frame is not None and self.data_frame2 is not None:
                    target=self.target_column
                    if encode_options == 'one-hot encoding':
                        for col in column:
                            self.data_frame = pd.get_dummies(self.data_frame, columns=[col])
                            self.data_frame2 = pd.get_dummies(self.data_frame2, columns=[col])


                    elif encode_options == 'target encoding':
                        enc = TargetEncoder(cols=column).fit(self.data_frame.drop(columns=target), self.data_frame.loc[:,target])
                        # transform the datasets
                        encode_data = enc.transform(self.data_frame.drop(columns=target))
                        encode_data2 = enc.transform(self.data_frame2)
                        self.data_frame = pd.concat([encode_data,self.data_frame.loc[:,self.target_column]],axis=1)
                        self.data_frame2 = encode_data2


                    elif encode_options == 'label encoding':
                        #LabelEncoder
                        for col in column:
                            labelencoder = LabelEncoder()
                            self.data_frame[col] = labelencoder.fit_transform(self.data_frame[col])
                            self.data_frame2[col] = labelencoder.transform(self.data_frame2[col])
                    
                    result=html.H5('success',style={'text-align': 'center','color':'green'})
                else:
                    false=html.H5('This column is a numeric variable.',style={'color': "red"})
                
            return result,false
# =======================================================================================================================
        
    #standardization
        page5_layout=html.Div(
        children=[
            html.Br(),
            html.H1(children="Feature Engineering",style={'text-align': 'center'}),
            html.Br(),
            html.H3("Standardization and Normalization",style={'text-align': 'center'}),
            html.Hr(),
            html.Br(),
            html.H5('You can choose one method to shape your data (optional)',style={'text-align': 'center'}),
            dcc.Dropdown(id="std-nor",
                            options=['No','Standardization','Normalization'],
                            multi=False,
                            value='No',
                            ),
            html.Br(),
            html.Div([html.Button('confirm', id='button_std',n_clicks=0,style={'text-align': 'center','background-color': 'rgb(190, 223, 210)'})],
                      style={'textAlign': 'center', 'position': 'relative'}),
            html.Br(),
            html.Div(id='success_std',style={'text-align': 'center'}),
        ])
        @app.callback(
        Output(component_id='success_std',component_property='children'),            
        [Input(component_id='std-nor',component_property='value'),
        Input(component_id='button_std', component_property='n_clicks')])
        
        def std(std_nor,n_clicks):
            result={}
            if n_clicks:   
                    if std_nor=='Standardization':
                        data=self.data_frame
                        data1=self.data_frame2
                        
                        for i in self.data_frame.columns:
                            if len(set(self.data_frame.loc[:,i]))<4 or self.data_frame.dtypes[i] in ['object'] or i=='id' or i==self.target_column:
                                data = data.drop([i], axis = 1)
                        for i in self.data_frame2.columns:
                            if len(set(self.data_frame2.loc[:,i]))<4 or self.data_frame2.dtypes[i] in ['object'] or i=='id'or i==self.target_column:
                                data1 = data1.drop([i], axis = 1)
                                
                        columns=data.columns
                        columns1=data1.columns
                        
                        data = preprocessing.scale(data)
                        data1 = preprocessing.scale(data1)
                        
                        data = pd.DataFrame(data)
                        data1 = pd.DataFrame(data1)
                        
                        data.columns=columns
                        data1.columns=columns1

                        for i in self.data_frame.columns:
                            if i in data.columns:
                                self.data_frame.loc[:,i]=data.loc[:,i]
                        for j in self.data_frame2.columns:
                            if j in data1.columns:
                                self.data_frame2.loc[:,j]=data1.loc[:,j]
                        result=html.H5('success',style={'text-align': 'center','color':'green'}) 
                       
                    elif std_nor=='Normalization':
                        data=self.data_frame
                        data1=self.data_frame2
                        
                        for i in self.data_frame.columns:
                            if len(set(self.data_frame.loc[:,i]))<4 or self.data_frame.dtypes[i] in ['object'] or i=='id' or i==self.target_column:
                                data = data.drop([i], axis = 1)
                        for i in self.data_frame2.columns:
                            if len(set(self.data_frame2.loc[:,i]))<4 or self.data_frame2.dtypes[i] in ['object'] or i=='id' or i==self.target_column:
                                data1 = data1.drop([i], axis = 1)
                        
                        columns=data.columns
                        columns1=data1.columns
                        data=preprocessing.normalize(data)
                        data1=preprocessing.normalize(data1)

                        data = pd.DataFrame(data)
                        data1 = pd.DataFrame(data1)
                        
                        data.columns=columns
                        data1.columns=columns1

                        for i in self.data_frame.columns:
                            if i in data.columns:
                                self.data_frame.loc[:,i]=data.loc[:,i]
                        for j in self.data_frame2.columns:
                            if j in data1.columns:
                                self.data_frame2.loc[:,j]=data1.loc[:,j]
                        result=html.H5('success',style={'text-align': 'center','color':'green'})       
            return result
            
# =======================================================================================================================

        page6_layout = html.Div(
            children=[
                html.Br(),
                html.H2("Introdution of methods for feature selection."),
                html.Br(), 
                html.H4("Filter methods"),
                html.Br(),
                html.H5("Filter methods evaluate the features independently of the model,\
                        using statistical tests or correlation measures to determine their relevance.\
                        They are computationally efficient and can handle high-dimensional datasets,\
                        but they may overlook interactions between features.",style={'text-align': 'left'}),
                html.Br(),
                html.H4("Wrapper methods"),
                html.Br(),
                html.H5("Wrapper methods evaluate subsets of features by training and testing a \
                        model with each subset, selecting the subset that performs the best. \
                        They can capture interactions between features but are computationally \
                        intensive and may overfit the data",style={'text-align': 'left'}),
                html.Br(),
                html.H4("Embedded methods"),
                html.Br(),
                html.H5("Embedded methods incorporate feature selection into the model training process,\
                        selecting the most relevant features during model training. \
                        They are computationally efficient and can capture interactions between features,\
                        but may not perform as well as wrapper methods on some datasets",style={'text-align': 'left'}),
                html.Br()
                
            ],
            style={'text-align': 'center','width':'90%'}
        )
        
# =======================================================================================================================        
        
        #filter method
        page7_layout = html.Div(
            children=[
                #特徵篩選
                html.Br(),
                html.H1(id='useless3',children="Feature Selection by Correlation",style={'text-align': 'center'}),
                html.Br(),
                html.H3(id='target-chosen',style={'text-align': 'left'}),
                html.Br(),
                html.H3("threshold for features",style={'text-align': 'left'}),
                html.H5("if the correlation of dependent variables are greater than this value, the less effective features would be droped.",
                         style={'text-align': 'left'}),
                
                dcc.Dropdown(id="threshold_for_features",
                            options=[{'label':str(num),'value':num} \
                                      for num in [round(x, 2) for x in list(np.arange(0.05, 1.00, 0.05))]],
                            multi=False,
                            value=0.95,
                            style={'width':"70%"}),
                html.Br(),
                html.H3("threshold for target",style={'text-align': 'left'}),
                html.H5("show the correlation of dependent and independent variables if the values are greater than this value.",
                         style={'text-align': 'left'}),
                dcc.Dropdown(id="threshold_for_target",
                             options=[{'label':str(num),'value':num} \
                                      for num in [round(x, 2) for x in list(np.arange(0.05, 1.00, 0.05))]],
                            multi=False,
                            value=0.6,
                            style={'width':"70%"}),
                dcc.Graph(id="selectheatmap",style={'display': 'block'}),
                html.H3('confirm if you want to use these feature',style={'text-align': 'center'}),
                html.Br(),
                html.Div([html.Button('confirm', id='button-filter',n_clicks=0,style={'text-align': 'center','background-color': 'rgb(190, 223, 210)'})],
                          style={'textAlign': 'center', 'position': 'relative'}),
                html.Br(),
                html.Div(id='result_filter'),
                html.Br()

            ]
        )
        @app.callback(
            Output(component_id='target-chosen', component_property='children'),
            Input(component_id='useless3', component_property='children'),
        )
        def target(useless3):
            return html.H3(f'The target you select before : {self.target_column}')
        
        @app.callback(
            Output(component_id='selectheatmap', component_property='figure'),
            Output(component_id='result_filter', component_property='children'),
            Input(component_id='threshold_for_target', component_property='value'),
            Input(component_id='threshold_for_features', component_property='value'),
            Input(component_id='button-filter', component_property='n_clicks')
        )

        def update_plotp4(threshold_for_target,threshold_for_features,n_clicks):
            def anyNone(*args):
                for i in args:
                    if i is None:
                        return True 
                return False
            #defalt
            fig4 = self.placeholder_figure()
            df_1 = self.data_frame
            result={}
            if not anyNone(df_1,threshold_for_target,threshold_for_features): 
                target=self.target_column
                target_corr = df_1.corr()[target]
                col_select=[]
                
                #找出跟target相關性高的column
                for i,v in zip(target_corr.index,target_corr.values):
                    if v > threshold_for_target:
                        col_select.append(i)
                        
                #用這些column做一個新的data
                select_data = df_1.loc[:,col_select]
                no_target_select_data = select_data.drop(columns=target,axis=1) 
                high_correlation=[]
                last=[i for i in select_data.columns]
                for col in no_target_select_data.corr():
                    high_correlation.append(col)
                    for index,value in zip(no_target_select_data.corr()[col].index,no_target_select_data.corr()[col].values): 
                        if abs(value) > threshold_for_features and value!=1:
                            high_correlation.append(index)
                    data_temperate = df_1.loc[:,high_correlation]
                    a=[data_temperate[i].corr(df_1.loc[:,target]) for i in data_temperate]
                    b=[i for i in data_temperate]
                    for num in range(len(a)):
                        if a[num]==max(a):
                            del b[num]
                            for i in b:
                                if i in last:
                                    last.remove(i)
                    high_correlation=[]
                data=df_1.loc[:,last]
                
                if n_clicks:
                    self.data_frame=df_1.loc[:,last]
                    last.remove(target)
                    self.data_frame2=self.data_frame2.loc[:,last]
                    result=html.H5('success',style={'text-align': 'center','color':'green'})
                
                correlation=round(data.corr(),2)
                x = list(correlation.index)
                y = list(correlation.columns)
                z = np.array(correlation)

                fig4 = ff.create_annotated_heatmap(z, x = x, y = y , annotation_text = np.around(z, decimals=2),
                                                   hoverinfo='z',colorbar_thickness=30, colorbar_ticklen=3,
                                                   colorscale=[[0.0, 'rgb(221, 239, 232)'], [0.5, 'rgb(159, 208, 200)'], [1.0, 'rgb(82, 133, 109)']])
                fig4.update_layout(title_text='<b>Correlation Matrix (cont. features)<b>',
                    title_x=0.3,titlefont={'size': 24}, xaxis_showgrid=False, xaxis={'side': 'bottom'},
                    yaxis_showgrid=False, yaxis_autorange='reversed', paper_bgcolor=None,)
                
            return fig4,result 
            
            
# =======================================================================================================================

        page8_layout = html.Div(
            children=[
                html.Br(),
                
                #竭盡式特徵選取法
                html.H1(id='uselesswrapper',children="Feature Selection by Exhaustive Feature Selector",style={'text-align': 'center'}),
                html.Br(),
                html.Div(id="warning",style={'color': "rgb(220, 121, 106)","font-size": "22px"}),

                #勾選框可以先篩掉確定不要的特徵(選要的)
                html.H3("Select the features to be included in the wrapper method:",style={'text-align': 'center'}),
                dcc.Checklist(id="option_for_wrapper",
                    labelStyle={"display": "inline-block", "margin-right": "12px"},
                    inline=True,
                    style={'text-align': 'center'}),
                html.Br(),
                
                html.H3("Minimum number of features:",style={'text-align': 'left'}),
                dcc.Dropdown(id="min_f",
                            
                            multi=False,
                            style={'width':"70%"}),
                html.Br(),
                html.H3("Maximum number of features:",style={'text-align': 'left'}),
                dcc.Dropdown(id="max_f",
                            multi=False,
                            style={'width':"70%"}),
                html.Br(),
                
                #等使用者調整好所有項目再開始update(讓他按按鈕)
                html.Div([  html.Button('Ready!', id='update-button', n_clicks=0,
                            style={'background-color': 'rgb(190, 223, 210)','font-size': '16px'})],
                            style={'text-align': 'center'}),

                html.Br(),
                html.Br(),
                html.H3("Feature selected by EFS :",style={'text-align': 'center'}),
                dbc.Spinner(html.Div(id="efs_result",style={'text-align': 'center'}),color="primary"),
                html.Br(),
                html.H3('confirm if you want to use these feature',style={'text-align': 'center'}),
                html.Br(),
                html.Div([  html.Button('confirm!', id='decide-button', n_clicks=0,
                            style={'background-color': 'rgb(190, 223, 210)','font-size': '16px'})],
                            style={'text-align': 'center'}),
                html.Br(),
                html.H3(id="confirm_result",style={'text-align': 'center'}),
                html.Br(),
                ]
        )
        
        @app.callback(
            Output("min_f", "options"),
            Output("max_f", "options"),
            Output("option_for_wrapper", "options"),
            Input("uselesswrapper", "children")
        )
        def get_options(useless):
            if self.data_frame2 is not None:
                dff = self.data_frame2
                output = ['Select All']+list(dff.columns)+['Deselect All']
                return  list(range(1,len(dff.columns))),list(range(1,len(dff.columns))),output
            else:
                return  [],[],{}
            
        @app.callback(
        Output("option_for_wrapper", "value"),
        Input("option_for_wrapper", "options"),
        Input("option_for_wrapper", "value")
        )
        def select_all_options(options, value):
            if options is None:
                return []
            if value is None:
                return [option for option in options if option != "Deselect All"]
            elif "Deselect All" in value:
                return []
            elif "Select All" in value:
                return [option for option in options if option != "Deselect All"]
            else:
                return value
    
        @app.callback(
            Output(component_id='warning', component_property='children'),
            Output(component_id='efs_result', component_property='children'),
            Output(component_id='confirm_result', component_property='children'),
            Input(component_id='option_for_wrapper', component_property='value'),
            Input(component_id='min_f', component_property='value'),
            Input(component_id='max_f', component_property='value'),
            Input(component_id='update-button', component_property='n_clicks'),
            Input(component_id='decide-button', component_property='n_clicks'),
        )

        def update_plotp5(wanted,min_f,max_f,n_clicks,n_clicks_decide):
            def anyNone(*args):
                for i in args:
                    if i is None:
                        return True
            warn = {}
            result = {}
            confirm_result = {}
            targetdf = None if (self.data_frame is None) else (self.data_frame)
            target = None if (self.target_column is None) else (self.target_column)
            if targetdf is None : 
                warn = html.H5(f"You haven't upload train and test data",style={'text-align': 'center','color':'red'})
            if not anyNone(targetdf,target,wanted,min_f,max_f) and target in targetdf.columns:  
                missing_values = targetdf.isna().any().any()
                if missing_values:
                    warn = "Please go to 'Missing Value Imputing' to fill in missing values first!"
                else:
                    warn=''
                    if not isinstance(min_f, int) or not isinstance(max_f, int) or len(wanted) == 0:
                        result='Please check once again that you have set all the parameters.'
                    elif n_clicks: 
                        num_features = len(wanted)
                        if  min_f > max_f :
                            result = "The maximum number of features should be greater than or equal to the minimum one."
                        elif num_features < max_f :
                            result = "Please select more than {} features".format(max_f)
                        else:
                            if 'Select All' in wanted:
                                    x = self.data_frame.drop(columns=target) #全部
                                    y = self.data_frame.loc[:,target]
                                    selector = ExhaustiveFeatureSelector(
                                        LinearRegression(), 
                                        min_features=min_f,
                                        max_features=max_f,
                                        scoring='neg_mean_squared_error',
                                        print_progress=True,cv=5)

                                    selector = selector.fit(x, y)

                                    result_list = selector.best_feature_names_
                                    result = ' / '.join(result_list)
                                    if n_clicks_decide:
                                        self.data_frame=self.data_frame.loc[:,list(result_list)+[target]]
                                        self.data_frame2=self.data_frame2.loc[:,result_list]
                                        confirm_result = html.H5('success',style={'text-align': 'center','color':'green'})
                            elif 'Deselect All' in wanted:
                                result = 'Please at least choose one feature.'
                            else:
                                x = self.data_frame.loc[:,wanted] #取出想要的欄位
                                y = self.data_frame.loc[:,target]

                                selector = ExhaustiveFeatureSelector(
                                    LinearRegression(), 
                                    min_features=min_f,
                                    max_features=max_f,
                                    scoring='neg_mean_squared_error',
                                    print_progress=True,cv=5)

                                selector = selector.fit(x, y)

                                result_list = selector.best_feature_names_
                                result = ' / '.join(result_list)
                                if n_clicks_decide:
                                    self.data_frame=self.data_frame.loc[:,list(result_list)+[target]]
                                    self.data_frame2=self.data_frame2.loc[:,result_list]
                                    confirm_result = html.H5('success',style={'text-align': 'center','color':'green'})
                    else:
                        result = "Click the Ready button to let me know that you have set the parameters."
            return warn, result, confirm_result
            
# ======================================================================================================================= 
            
        page9_layout = html.Div(
            children=[
                html.Br(),
                html.H1(children="Embedded Methods",style={'text-align': 'center'}),
                html.Br(),
                html.H4('Which method do you want to apply?',style={'text-align': 'center'}),
                html.Br(),
                
                html.Div([dcc.Dropdown(id="method_for_emb",
                            options=[{'label':'Lasso','value':'Lasso'}, 
                                     {'label':'Ridge','value':'Ridge'},
                                     {'label':'Elastic nets','value':'Elastic nets'},
                                     {'label':'Tree-base','value':'Tree-base'}],
                            multi=False,value='Lasso',style={'width':"75%"})],style={'display': 'flex', 'justify-content': 'center', 'align-items': 'center','margin-left': '13%'}),
                
                html.Br(),
                #結果
                html.Br(),
                html.Div(id="illustration_for_restrict",style={'text-align': 'center'}),
                html.Br(),
                
                html.Div([dcc.Dropdown(id="display_restrict",multi=False,
                                       style={'width':"75%"})],style={'display': 'flex', 'justify-content': 'center', 'align-items': 'center','margin-left': '13%'}),
                
                html.Br(),
                dbc.Spinner(html.Div(id='output_embresult', style={'overflow':'auto','width':"50%",'text-align': 'center'}),color="primary"),
                html.Br(),
                html.H3('confirm if you want to use these feature',style={'text-align': 'center'}),
                html.Br(),
                html.Div([  html.Button('confirm!', id='decide', n_clicks=0,
                            style={'background-color': 'rgb(190, 223, 210)','font-size': '16px'})],
                            style={'text-align': 'center'}),
                html.Br(),
                html.H3(id="confirm_",style={'text-align': 'center','color':'green'}),
                html.Br()
                ],
            style={'text-align': 'left'}      
        )
        
        
        @app.callback(
            Output("illustration_for_restrict", "children"),
            Input("method_for_emb", "value")
        )
        
        def update_options(method_for_emb):
            illustration_for_restrict = html.H5('you have not upload your data.',style={'color':'red'})
            targetdf = None if (self.data_frame is None) else (self.data_frame)
            if targetdf is not None:
                if method_for_emb =='Lasso':
                    illustration_for_restrict = html.H4('Do you want to remove features with coefficient equal to 0?')
                else:
                    illustration_for_restrict = html.H4('What is the threshold for the coefficient values \
                    below which you would like to exclude features from the table?')
            return illustration_for_restrict
        
        @app.callback(
            Output('display_restrict','options'),
            Input("method_for_emb", "value")
        )
        
        def update_no_target(method_for_emb):
            options=[{"label": 'select', "value": 'select'}]
            targetdf = None if (self.data_frame is None) else (self.data_frame)
            if targetdf is not None:
                if method_for_emb =='Lasso':
                    options = [{'label':'Yes','value':'Yes'}, 
                             {'label':'No','value':'No'}]
                else:
                    options=[{'label':str(num),'value':num} \
                                       for num in [round(x, 3) for x in list(np.arange(0.01, 0.5, 0.01))]]
            return options
        
        @app.callback(
        Output(component_id='output_embresult', component_property='children'),
        Output(component_id='confirm_', component_property='children'),
        Input(component_id='method_for_emb', component_property='value'),
        Input(component_id='display_restrict', component_property='value'),
        Input(component_id='decide', component_property='n_clicks')
            
        )

        def update_plotp7(method_for_emb,display_restrict,n_clicks):
            def anyNone(*args):
                for i in args:
                    if i is None:
                        return True
            # default value
            output_embresult = None 
            confirm_ = {}
            target=self.target_column
            targetdf = None if (self.data_frame  is None) else (self.data_frame )
            if not anyNone(method_for_emb,display_restrict) and display_restrict != 'select' :
                missing_values = targetdf.isna().any().any()
                feature_names = self.data_frame2.columns.tolist()
                if missing_values:
                    output_embresult = None
                else:
                    y = targetdf.loc[:,target]
                    x = targetdf.drop(target, axis=1)
                    if method_for_emb == 'Lasso':
                        reg=LassoCV()
                        reg.fit(x,y)
                        coef_df = pd.DataFrame({'Features': x.columns, 'Coefficients': reg.coef_})
                        if display_restrict == 'No':
                            output_embresult = dash_table.DataTable(data=coef_df.to_dict('records')) 
                        else:
                            nonzero_mask = (coef_df['Coefficients'] != 0)
                            temp = coef_df.loc[nonzero_mask]
                            feature_names = temp.loc[:, 'Features'].tolist()
                            output_embresult = dash_table.DataTable(data=temp.to_dict('records')) 
                    elif method_for_emb == 'Ridge':
                        ridgecv=RidgeCV()
                        ridgecv.fit(x,y)
                        coef_df = pd.DataFrame({'Features': x.columns, 'Coefficients': ridgecv.coef_})
                        temp = coef_df[coef_df['Coefficients'] >= display_restrict].reset_index(drop=True)
                        feature_names = temp.loc[:, 'Features'].tolist()
                        output_embresult = dash_table.DataTable(data=temp.to_dict('records')) 
                    elif method_for_emb == 'Elastic nets':
                        e_net = ElasticNet(alpha = 1)
                        e_net.fit(x, y)
                        ecoef_df = pd.DataFrame({'Features': x.columns, 'Coefficients': e_net.coef_})
                        temp = ecoef_df[ecoef_df['Coefficients'] >= display_restrict].reset_index(drop=True)
                        feature_names = temp.loc[:, 'Features'].tolist()
                        output_embresult = dash_table.DataTable(data=temp.to_dict('records')) 
                    elif method_for_emb == 'Tree-base':
                        model = RandomForestRegressor(n_estimators=340)
                        model.fit(x, y)
                        importances = model.feature_importances_
                        final_df = pd.DataFrame({"Features": x.columns, "Importances":importances})
                        final_df.set_index('Importances')
                        temp = final_df[final_df['Importances'] >= display_restrict].reset_index(drop=True)
                        feature_names = temp.loc[:, 'Features'].tolist()
                        output_embresult = dash_table.DataTable(data=temp.to_dict('records')) 
                    if n_clicks:
                        self.data_frame=self.data_frame.loc[:,list(feature_names)+[target]]
                        self.data_frame2=self.data_frame2.loc[:,feature_names]
                        confirm_ = html.H5('success',style={'text-align': 'center','color':'green'})
                    
            return output_embresult, confirm_
        
# =======================================================================================================================

        #第10頁   
        page10_layout = html.Div(
            children=[
                html.Br(),
                html.H1("Prediction",style={'text-align': 'center'}),
                html.Br(),
                html.Div(id="warn",style={'color': "rgb(220, 121, 106)","font-size": "22px"}),
                html.Br(),
                
                #選演算法
                html.H3("Select the desired algorithm:",style={'text-align': 'left'}),
                dcc.Dropdown(id="alg",
                            options=[{'label':'XGBRegressor','value':'XGBRegressor'},
                                     {'label':'LGBMRegressor','value':'LGBMRegressor'},
                                     {'label':'LinearRegressor','value':'LinearRegressor'},
                                     {'label':'GradientBoostingRegressor','value':'GradientBoostingRegressor'},],
                            multi=False,value='LinearRegressor',
                            style={'width':"70%"}),
                html.Br(),
                
                #問使用者要不要用PCA
                html.H3("If you want to conduct PCA decomposition, choose the n_components you want:",style={'text-align': 'left'}),
                dcc.Dropdown(id="PCA_com",
                             options=[
                                      {'label': '1', 'value': 1},
                                      {'label': '2', 'value': 2},
                                      {'label': '3', 'value': 3},
                                      {'label': '4', 'value': 4},
                                      {'label': '5', 'value': 5},
                                      {'label': 'No', 'value': 'No'}
                                     ],
                            multi=False,
                            value="No",
                            style={'width': "70%"}
                ),
                html.Br(),
                html.H3("If you want to use optuna to optimize hyperparameters",style={'text-align': 'left'}),
                html.Br(),
                dcc.RadioItems(id="optuna-radioitems",
                       options=["no", "yes"], value="no",
                       labelStyle={'display': 'inline-block', 'margin-right': '10px'}),
                
                #等使用者調整好所有項目再開始update(讓他按按鈕)
                html.Div([html.Button('Start Training!', id='train-button',n_clicks=0,style={'text-align': 'center','background-color': 'rgb(190, 223, 210)'})],
                      style={'textAlign': 'center', 'position': 'relative'}),
                html.Br(),
                dbc.Spinner(html.Div(id="warn",style={'text-align': 'center'}),color="primary"),
                html.Br(),
                
                #結果(train資料)呈現 某種分數之類的
                html.H3("Which metric would you like to use as the evaluation criterion?",style={'text-align': 'center'}),
                html.Div([dcc.Dropdown(id="metrics",
                             options=[
                                      {'label': 'RMSE', 'value':'RMSE'},
                                      {'label': 'R-squared', 'value':'R-squared'},
                                      {'label': 'MAE', 'value':'MAE'}
                                     ],
                            multi=False,
                            value='',
                            style={'width': '80%','text-align': 'center'}
                            )],style={'display': 'flex', 'justify-content': 'center', 'align-items': 'center','margin-left': '13%'}),
                
                html.Br(),
                html.Div(id='warnformodel',style={'text-align': 'center'}),
                html.Div(id='train_result',style={'text-align': 'center'}),
                html.Br(),
                
                #確認要用現有模型來預測 
                html.H3("click cofirm to make a prediction in test set",style={'text-align': 'center'}),
                html.Br(),
                html.Div([html.Button('confirm', id='yes-button',n_clicks=0,style={'text-align': 'center','background-color': 'rgb(190, 223, 210)'})],
                      style={'textAlign': 'center', 'position': 'relative'}),
                
                #結果table
                html.Br(),
                html.Br(),
                html.H3("Prediction result",style={'text-align': 'left'}),
                html.Br(),
                html.Div(dash_table.DataTable(id="pre_data",page_size=10,
                                              export_format='csv',
                                              export_headers='none'
                                              ),
                         style={'width': '50%','padding': '10px'},
                        ),
                html.Br()
            ]
        )
        
        @app.callback(
            Output(component_id='warn', component_property='children'),
            Output(component_id='warnformodel', component_property='children'),
            Input(component_id='alg', component_property='value'),
            Input(component_id='PCA_com', component_property='value'),
            Input(component_id='train-button', component_property='n_clicks'),
            Input(component_id="optuna-radioitems", component_property='value'),
            State(component_id='pre_data', component_property='data'), 
            State(component_id='pre_data', component_property='columns')
            
        )

        def update_plotp6(alg,PCA_com,n_clicks1,optunause,data,columns):
            def anyNone(*args):
                for i in args:
                    if i is None:
                        return True
                    
            # default value
            targetdf = None if (self.data_frame is None) else (self.data_frame)
            testdf = None if (self.data_frame2 is None) else (self.data_frame2)
            target = self.target_column
            np.set_printoptions(precision=2, suppress=True)
            result = None
            warn = html.H5("you haven't upload your data.",style={'color': "red"})
            outcome='please do the prediction first.'
            if not anyNone(target,targetdf,testdf,alg,PCA_com) and target in targetdf.columns:
                result = np.zeros((testdf.shape[0], 2))
                result[:, 0] = [i for i in range(testdf.shape[0])] 
                result[:, 1] = 0
                result = pd.DataFrame(result)
                result = result.to_dict('records')
                
                #如果訓練集有缺失資料 
                missing_values = self.data_frame.isna().any().any()
                if missing_values:
                    warn = html.H5("Please go to 'Missing Value Imputing' to fill in missing values first!",style={'color': "red"})
                    outcome =html.H5("Click the 'Start Training' button to let me know that you have set the parameters.",style={'color': "red"})
                else: #確定沒有缺失值
                    if not n_clicks1: #還沒按開始訓練
                        warn = html.H5("Click the 'Start Training' button to let me know that you have set the parameters.",style={'color': "red"})
                    elif PCA_com != 'No':
                        if int(PCA_com) > len(targetdf.columns)-1: #PCA降維後的特徵數大於所選for training的特徵數
                            outcome = html.H5("The number of PCA components cannot exceed the number of selected features.",style={'color': "red"})
                        else:
                            outcome={}
                            if alg == 'LinearRegressor':
                                #LR with PCA
                                x = self.data_frame.drop(columns=target) #取出想要的欄位
                                y = self.data_frame.loc[:,target]
                                x_test=self.data_frame2.loc[:,x.columns]  
                                pca = PCA(n_components=PCA_com)
                                pca.fit(x)
                                x = pca.transform(x)
                                x_train,x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)
                                if optunause =="yes":
                                    #optuna
                                    def objective_lr(trial):
                                        params = {
                                                 'n_jobs': trial.suggest_int('n_jobs', 1, 5),
                                                 'fit_intercept': trial.suggest_categorical('fit_intercept', [True,False]),
                                                 'copy_X': trial.suggest_categorical('copy_X', [True,False]),
                                                 'positive': trial.suggest_categorical('positive', [True,False]),
                                             }
                                        reg = LinearRegression(**params)
                                        reg.fit(x_train, y_train)
                                        return mean_squared_error(y_valid, reg.predict(x_valid), squared=False)
                                    study = optuna.create_study(direction='minimize')
                                    study.optimize(objective_lr, n_trials = 20)
                                    #fit
                                    model=LinearRegression(**study.best_trial.params)
                                else:
                                    model=LinearRegression()
                                model.fit(x_train,y_train)
                                
                                #模型表現
                                y_pred=model.predict(x_valid)
                                rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
                                score = model.score(x_train, y_train)
                                mae = mean_absolute_error(y_valid, y_pred)
                                self.train_outcome=[rmse,score,mae]
                                x_test = pca.transform(x_test)
                                result=model.predict(x_test)
                                result = pd.DataFrame(result,columns=["Result"])
                                result.insert(loc=0, column='Index', value=[i for i in range(self.data_frame2.shape[0])])
                                self.result = result.to_dict('records')
                                
                            elif alg == 'XGBRegressor':
                                x = self.data_frame.drop(columns=target) #取出想要的欄位
                                y = self.data_frame.loc[:,target]
                                x_test=self.data_frame2.loc[:,x.columns]
                                pca = PCA(n_components=PCA_com)
                                pca.fit(x)
                                x = pca.transform(x)
                                x_train,x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)
                                if optunause =="yes":
                                    
                                    #optuna
                                    def objective_xgb(trial):
                                        params = {
                                                 'max_depth': trial.suggest_int('max_depth', 3, 10),
                                                 "subsample": trial.suggest_float("subsample", 0.2, 1.0),
                                                 'n_estimators': trial.suggest_int('n_estimators', 100, 500, 25),
                                                 'eta': trial.suggest_float("eta", 1e-8, 1.0, log=True),
                                                 'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
                                                 'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
                                                 'gamma': trial.suggest_float("gamma", 1e-8, 1.0, log=True),
                                                 'min_child_weight': trial.suggest_int('min_child_weight', 2, 10),
                                                 'grow_policy': trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
                                                 "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0)
                                             }
                                        reg = XGBRegressor(**params, random_state=42)
                                        reg.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], eval_metric='rmse', verbose=True)
                                        return mean_squared_error(y_valid, reg.predict(x_valid), squared=False)
                                    study = optuna.create_study(direction='minimize')
                                    study.optimize(objective_xgb, n_trials = 20)
                                    
                                    #fit
                                    model=XGBRegressor(**study.best_trial.params,random_state=42)
                                else:
                                    model=XGBRegressor()
                                model.fit(x_train,y_train)
                                
                                #模型表現
                                y_pred=model.predict(x_valid)
                                rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
                                score = model.score(x_train, y_train)
                                mae = mean_absolute_error(y_valid, y_pred)
                                self.train_outcome=[rmse,score,mae]
                                x_test = pca.transform(x_test)
                                result=model.predict(x_test)
                                result = pd.DataFrame(result,columns=["Result"])
                                result.insert(loc=0, column='Index', value=[i for i in range(self.data_frame2.shape[0])])
                                self.result = result.to_dict('records')
                            elif alg == 'LGBMRegressor':
                                x = self.data_frame.drop(columns=target) #取出想要的欄位
                                y = self.data_frame.loc[:,target]
                                x_test=self.data_frame2.loc[:,x.columns]
                                pca = PCA(n_components=PCA_com)
                                pca.fit(x)
                                x = pca.transform(x)
                                x_train,x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)
                                if optunause =="yes":
                                    
                                    #optuna
                                    def objective_lgb(trial):
                                        params = {
                                                'n_estimators': trial.suggest_int('n_estimators', 100,500,25),
                                                'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 1),
                                                'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 1),
                                                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
                                                'subsample': trial.suggest_float('subsample', 0.2, 1.0),
                                                'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
                                                'max_depth': trial.suggest_int('max_depth', 2, 30),
                                                'num_leaves' : trial.suggest_int('num_leaves', 2, 1000),
                                                'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
                                                'cat_smooth' : trial.suggest_int('min_data_per_groups', 10, 100)
                                             }
                                        reg = lgb.LGBMRegressor(**params, random_state=42)
                                        reg.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], eval_metric='rmse', verbose=True)
                                        return mean_squared_error(y_valid, reg.predict(x_valid), squared=False)
                                    study = optuna.create_study(direction='minimize')
                                    study.optimize(objective_lgb, n_trials = 20)
                                    
                                    #fit
                                    model=lgb.LGBMRegressor(**study.best_trial.params)
                                else:
                                    model=lgb.LGBMRegressor()
                                model.fit(x_train,y_train)
                                
                                #模型表現
                                y_pred=model.predict(x_valid)
                                rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
                                score = model.score(x_train, y_train)
                                mae = mean_absolute_error(y_valid, y_pred)
                                self.train_outcome=[rmse,score,mae]
                                
                                x_test = pca.transform(x_test)
                                result=model.predict(x_test)
                                result = pd.DataFrame(result,columns=["Result"])
                                result.insert(loc=0, column='Index', value=[i for i in range(self.data_frame2.shape[0])])
                                self.result = result.to_dict('records')
                            elif alg == 'GradientBoostingRegressor':
                                x = self.data_frame.drop(columns=target) #取出想要的欄位
                                y = self.data_frame.loc[:,target]
                                x_test=self.data_frame2.loc[:,x.columns]
                                pca = PCA(n_components=PCA_com)
                                pca.fit(x)
                                x = pca.transform(x)
                                x_train,x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)
                                if optunause =="yes":
                                    
                                    #optuna
                                    def objective_gbdt(trial):
                                        params = {
                                                'n_estimators': trial.suggest_int('n_estimators', 100, 500, 25),
                                                'max_features':trial.suggest_int("max_features", 10,50,5),
                                                'subsample': trial.suggest_float('subsample', 0.2, 1.0),
                                                'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
                                                'max_depth': trial.suggest_int('max_depth', 2, 30),
                                                "min_samples_split": trial.suggest_int('min_samples_split', 2, 7),
                                             }
                                        reg = GradientBoostingRegressor(**params, random_state=42)
                                        reg.fit(x_train, y_train)
                                        return mean_squared_error(y_valid, reg.predict(x_valid), squared=False)
                                    study = optuna.create_study(direction='minimize')
                                    study.optimize(objective_gbdt, n_trials = 20)
                                    
                                    #fit
                                    model=GradientBoostingRegressor(**study.best_trial.params)
                                else:
                                    model=GradientBoostingRegressor()
                                model.fit(x_train,y_train)
                                
                                #模型表現
                                y_pred=model.predict(x_valid)
                                rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
                                score = model.score(x_train, y_train)
                                mae = mean_absolute_error(y_valid, y_pred)
                                self.train_outcome=[rmse,score,mae]
                                x_test = pca.transform(x_test)
                                result=model.predict(x_test)
                                result = pd.DataFrame(result,columns=["Result"])
                                result.insert(loc=0, column='Index', value=[i for i in range(self.data_frame2.shape[0])])
                                self.result = result.to_dict('records')
                            warn=html.H5("training complete!!",style={'color': "green"})            
                    else:
                        outcome={}
                        if alg == 'LinearRegressor':
                            x = self.data_frame.drop(columns=target) #取出想要的欄位
                            y = self.data_frame.loc[:,target]
                            x_test=self.data_frame2.loc[:,x.columns]
                            x_train,x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)
                            if optunause =="yes":
                                
                                #optuna
                                def objective_lr(trial):
                                    params = {
                                             'n_jobs': trial.suggest_int('n_jobs', 1, 5),
                                             'fit_intercept': trial.suggest_categorical('fit_intercept', [True,False]),
                                             'copy_X': trial.suggest_categorical('copy_X', [True,False]),
                                             'positive': trial.suggest_categorical('positive', [True,False]),
                                         }
                                    reg = LinearRegression(**params)
                                    reg.fit(x_train, y_train)
                                    return mean_squared_error(y_valid, reg.predict(x_valid), squared=False)
                                study = optuna.create_study(direction='minimize')
                                study.optimize(objective_lr, n_trials = 20)
                                #fit
                                model=LinearRegression(**study.best_trial.params)
                            else:
                                model=LinearRegression()
                            model.fit(x_train,y_train)
                            
                            #模型表現
                            y_pred=model.predict(x_valid)
                            rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
                            score = model.score(x_train, y_train)
                            mae = mean_absolute_error(y_valid, y_pred)
                            self.train_outcome=[rmse,score,mae]
                            result=model.predict(x_test)
                            result = pd.DataFrame(result,columns=["Result"])
                            result.insert(loc=0, column='Index', value=[i for i in range(self.data_frame2.shape[0])])
                            self.result = result.to_dict('records')
                        elif alg == 'XGBRegressor':
                            x = self.data_frame.drop(columns=target) #取出想要的欄位
                            y = self.data_frame.loc[:,target]
                            x_test=self.data_frame2.loc[:,x.columns]
                            x_train,x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)
                            if optunause =="yes":
                                
                                #optuna
                                def objective_xgb(trial):
                                    params = {
                                             'max_depth': trial.suggest_int('max_depth', 3, 10),
                                             "subsample": trial.suggest_float("subsample", 0.2, 1.0),
                                             'n_estimators': trial.suggest_int('n_estimators', 100, 500, 25),
                                             'eta': trial.suggest_float("eta", 1e-8, 1.0, log=True),
                                             'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
                                             'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
                                             'gamma': trial.suggest_float("gamma", 1e-8, 1.0, log=True),
                                             'min_child_weight': trial.suggest_int('min_child_weight', 2, 10),
                                             'grow_policy': trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
                                             "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0)
                                         }
                                    reg = XGBRegressor(**params, random_state=42)
                                    reg.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], eval_metric='rmse', verbose=True)
                                    return mean_squared_error(y_valid, reg.predict(x_valid), squared=False)
                                study = optuna.create_study(direction='minimize')
                                study.optimize(objective_xgb, n_trials = 20)
                                
                                #fit
                                model=XGBRegressor(**study.best_trial.params,random_state=42)
                            else:
                                model=XGBRegressor()
                            model.fit(x_train,y_train)
                            
                            #模型表現
                            y_pred=model.predict(x_valid)
                            rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
                            score = model.score(x_train, y_train)
                            mae = mean_absolute_error(y_valid, y_pred)
                            self.train_outcome=[rmse,score,mae]
                            result=model.predict(x_test)
                            result = pd.DataFrame(result,columns=["Result"])
                            result.insert(loc=0, column='Index', value=[i for i in range(self.data_frame2.shape[0])])
                            self.result = result.to_dict('records')
                        elif alg == 'GradientBoostingRegressor':
                            x = self.data_frame.drop(columns=target) #取出想要的欄位
                            y = self.data_frame.loc[:,target]
                            x_test=self.data_frame2.loc[:,x.columns]
                            x_train,x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)
                            if optunause =="yes":
                                
                                #optuna
                                def objective_gbdt(trial):
                                    params = {
                                            'n_estimators': trial.suggest_int('n_estimators', 100, 500, 25),
                                            'max_features':trial.suggest_int("max_features", 10,50,5),
                                            'subsample': trial.suggest_float('subsample', 0.2, 1.0),
                                            'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
                                            'max_depth': trial.suggest_int('max_depth', 2, 30),
                                            "min_samples_split": trial.suggest_int('min_samples_split', 2, 7),
                                         }
                                    reg = GradientBoostingRegressor(**params, random_state=42)
                                    reg.fit(x_train, y_train)
                                    return mean_squared_error(y_valid, reg.predict(x_valid), squared=False)
                                study = optuna.create_study(direction='minimize')
                                study.optimize(objective_gbdt, n_trials = 20)
                                
                                #fit
                                model=GradientBoostingRegressor(**study.best_trial.params)
                            else:
                                model=GradientBoostingRegressor()
                            model.fit(x_train,y_train)
                            
                            #模型表現
                            y_pred=model.predict(x_valid)
                            rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
                            score = model.score(x_train, y_train)
                            mae = mean_absolute_error(y_valid, y_pred)
                            self.train_outcome=[rmse,score,mae]
                            
                            result=model.predict(x_test)
                            result = pd.DataFrame(result,columns=["Result"])
                            result.insert(loc=0, column='Index', value=[i for i in range(self.data_frame2.shape[0])])
                            self.result = result.to_dict('records')
                        elif alg == 'LGBMRegressor':
                            x = self.data_frame.drop(columns=target) #取出想要的欄位
                            y = self.data_frame.loc[:,target]
                            x_test=self.data_frame2.loc[:,x.columns]
                            x_train,x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)
                            if optunause =="yes":
                                
                                #optuna
                                def objective_lgb(trial):
                                    params = {
                                            'n_estimators': trial.suggest_int('n_estimators', 100,500,25),
                                            'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 1),
                                            'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 1),
                                            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
                                            'subsample': trial.suggest_float('subsample', 0.2, 1.0),
                                            'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
                                            'max_depth': trial.suggest_int('max_depth', 2, 30),
                                            'num_leaves' : trial.suggest_int('num_leaves', 2, 1000),
                                            'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
                                            'cat_smooth' : trial.suggest_int('min_data_per_groups', 10, 100)
                                         }
                                    reg = lgb.LGBMRegressor(**params, random_state=42)
                                    reg.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], eval_metric='rmse', verbose=True)
                                    return mean_squared_error(y_valid, reg.predict(x_valid), squared=False)
                                study = optuna.create_study(direction='minimize')
                                study.optimize(objective_lgb, n_trials = 20)
                                
                                #fit
                                model=lgb.LGBMRegressor(**study.best_trial.params)
                            else:
                                model=lgb.LGBMRegressor()
                            model.fit(x_train,y_train)
                            
                            #模型表現
                            y_pred=model.predict(x_valid)
                            rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
                            score = model.score(x_train, y_train)
                            mae = mean_absolute_error(y_valid, y_pred)
                            self.train_outcome=[rmse,score,mae]
                            
                            result=model.predict(x_test)
                            result = pd.DataFrame(result,columns=["Result"])
                            result.insert(loc=0, column='Index', value=[i for i in range(self.data_frame2.shape[0])])
                            self.result = result.to_dict('records')
                        warn=html.H5("training complete!!",style={'color': "green"})
            return warn,outcome
        @app.callback(
            Output(component_id='train_result', component_property='children'),
            Input(component_id='metrics', component_property='value'),
        )    
        def metrics(metrics):
            if self.train_outcome is not None:
                if metrics == 'RMSE':
                    outcome = f'RMSE:{self.train_outcome[0]:.3f}'
                elif metrics == 'R-squared':
                    outcome = f'R-squared:{self.train_outcome[1]:.3f}'
                elif metrics == 'MAE':
                    outcome = f'MAE:{self.train_outcome[2]:.3f}'
                else:
                    outcome={}
            else:
                outcome={}
            return outcome

        @app.callback(
            Output(component_id='pre_data', component_property='data'),
            Input(component_id='yes-button', component_property='n_clicks'),
        )    
        def metrics(n_clicks2):
            if n_clicks2:
                result=self.result
            else:
                if self.data_frame2 is not None and self.data_frame is not None:
                    result = np.zeros((self.data_frame2.shape[0], 2))
                    result[:, 0] = [i for i in range(self.data_frame2.shape[0])] 
                    result[:, 1] = 0
                    result = pd.DataFrame(result)
                    result = result.to_dict('records')
                    
                else:
                    result = np.zeros((10, 2))
                    result[:, 0] = [i for i in range(10)] 
                    result[:, 1] = 0
                    result = pd.DataFrame(result)
                    result = result.to_dict('records')
            return result
# =======================================================================================================================
        @app.callback(
            Output('page-content', 'children'),
            [Input('url', 'pathname')])
    
        def display_page(pathname):
            if pathname == '/page0':
                return page0_layout
            elif pathname == '/page1':
                return page1_layout
            elif pathname == '/page2':
                return page2_layout
            elif pathname == '/page3':
                return page3_layout
            elif pathname == '/page4':
                return page4_layout
            elif pathname == '/page5':
                return page5_layout
            elif pathname == '/page6':
                return page6_layout
            elif pathname == '/page7':
                return page7_layout
            elif pathname == '/page8':
                return page8_layout
            elif pathname == '/page9':
                return page9_layout
            elif pathname == '/page10':
                return page10_layout
            else:
                return page0_layout
            
            
        app.run_server(debug=True,use_reloader=False,port=5674)


In [3]:
MyDash2().run_dash()

Dash is running on http://127.0.0.1:5674/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
